In [ ]:
from flask import Flask, request

app = Flask(__name__)

@app.route("/Teste", methods=["GET", "POST", "PUT", "DELETE"])
def hello_world():
    return {"Resultado": "Sucesso!"}


app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000


Press CTRL+C to quit
127.0.0.1 - - [22/Jul/2024 19:26:15] "GET / HTTP/1.1" 404 -
127.0.0.1 - - [22/Jul/2024 19:26:20] "GET /teste HTTP/1.1" 404 -
127.0.0.1 - - [22/Jul/2024 19:26:28] "GET /Teste HTTP/1.1" 200 -


In [ ]:
#Código para postar no replit:

from flask import Flask, jsonify, request
from flask_jwt_extended import (
    JWTManager,
    get_jwt,
    create_access_token,
    jwt_required,
)

vendas = {"filial_BH": 20000, "filial_SP": 80000, "filial_RJ": 40000}

usuarios = {"Alex": "4321"}

app = Flask(__name__)
app.config["JWT_SECRET_KEY"] = "sua_chave_secreta"
jwt = JWTManager(app)


@app.route("/login", methods=["POST"])
def login():
    usuario = request.json.get('usuario', None)
    senha = request.json.get('senha', None)

    if "admin" in usuario:
        permissoes = {"delete": "True",
                      "update": "True",
                      "read": "True",
                      "create": "True"
                     }
    else:
        permissoes = {"read": "True"}

    if usuario not in usuarios or senha != usuarios[usuario]:
        return jsonify({"msg": "Usuário ou senha incorretos"})

    access_token = create_access_token(identity=usuario, additional_claims=permissoes)
    return jsonify(access_token=access_token)



@app.route("/")
def consultar_vendas():
    return vendas

@app.route("/teste")
def teste():
    return get_jwt()

@app.route("/nova_filial", methods=["POST"])
@jwt_required()
def criar():
    if request.method == "POST":
        dados = request.form.to_dict()
        vendas.update(dados)
        return get_jwt()

@app.route("/editar/<filial>", methods=["PUT","DELETE"])
@jwt_required()
def editar(filial):
    if filial in vendas:
        if request.method == "PUT":
            novos_dados = request.form.to_dict()
            vendas[filial] = int(novos_dados.get('novo_valor'))
            return vendas
        elif request.method == "DELETE":
            del(vendas[filial])
            return vendas
    else:
        return {"erro": "Filial não encontrada"}

app.run(host = "0.0.0.0")

In [ ]:
import requests, json

nova_filial = {"filial_CTB": 150000}

req = requests.get("https://b2977271-5071-447a-af9e-cf61d47111bc-00-338m4902a998z.kirk.replit.dev/teste")

print(req.text)

<!doctype html>
<html lang=en>
<title>500 Internal Server Error</title>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>



In [ ]:
import os.path
from pprint import pprint

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/spreadsheets"]

# The ID and range of a sample spreadsheet.
SAMPLE_SPREADSHEET_ID = "00aKojOddSX00yVt0BYP0NRlihlUHvOrtNx0sSCng0Po"
SAMPLE_RANGE_NAME = "Página1!A1:B13"


def main():
  """Shows basic usage of the Sheets API.
  Prints values from a sample spreadsheet.
  """
  creds = None
  # The file token.json stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time.
  if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES)
  # If there are no (valid) credentials available, let the user log in.
  if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
      creds.refresh(Request())
    else:
      flow = InstalledAppFlow.from_client_secrets_file(
          "credentials.json", SCOPES
      )
      creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
      token.write(creds.to_json())


  try:
    service = build("sheets", "v4", credentials=creds)

    # Ler planilha
    sheet = service.spreadsheets()
    result = (
        sheet.values()
        .get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=SAMPLE_RANGE_NAME)
        .execute()
    )
    values = result.get("values") #Por padrão, todos os valores retornados são uma strin


    # Editar planilha
    i = 2
    #Inserindo valores por fórmula
    for linha in values[1:]:
        valores_adicionar = [
        [f"=B{i}*10%"]
        ]

    #Calculando direto no código para inserir somente valores na planilha:
    for linha in values[1:]:
        num = linha[1].replace("R$","").replace(" ","").replace(".","").replace(",",".")
        valores_adicionar = [
        [float(num) * 0.1]
        ]

        result = (
            sheet.values()
            .update(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=f"C{i}", valueInputOption = "USER_ENTERED", body={"values": valores_adicionar})
            .execute()
        )

        i += 1

    pprint(values)

  except HttpError as err:
    print(err)


if __name__ == "__main__":
  main()

[['Mês', 'Vendas'],
 ['janeiro', 'R$ 50.668,00'],
 ['fevereiro', 'R$ 98.475,00'],
 ['março', 'R$ 92.030,00'],
 ['abril', 'R$ 63.546,00'],
 ['maio', 'R$ 24.496,00'],
 ['junho', 'R$ 69.997,00'],
 ['julho', 'R$ 35.242,00'],
 ['agosto', 'R$ 73.105,00'],
 ['setembro', 'R$ 57.397,00'],
 ['outubro', 'R$ 30.293,00'],
 ['novembro', 'R$ 17.345,00'],
 ['dezembro', 'R$ 127.300,00']]


In [ ]:
import requests, json

url = "https://myfirstproject-1st-default-rtdb.firebaseio.com"

dados = {"cliente": "Lira", "produto": "fone de ouvido", "quantidade": 1, "valor": 50}

req = requests.post(f"{url}/Vendas/.json", data=json.dumps(dados))

print(req)
print(req.text)

<Response [200]>
{"name":"-O2oo7Snwopj6oh0ucaz"}


In [ ]:
dados = {"cliente": "Alon"}
req = requests.patch(f"{url}/Vendas/ID1/.json", data=json.dumps(dados))

print(req)
print(req.text)

<Response [200]>
{"cliente":"Alon"}


In [ ]:
req = requests.get(f"{url}/Vendas/.json")
dic_vendas = req.json()

compras_lira = [dic_vendas[venda] for venda in dic_vendas if dic_vendas[venda]["cliente"] == "Lira"]

print(compras_lira)

[{'cliente': 'Lira', 'produto': 'fone de ouvido', 'quantidade': 1, 'valor': 50}, {'cliente': 'Lira', 'produto': 'teclado', 'quantidade': 1, 'valor': 100}]


In [ ]:
req = requests.delete(f"{url}/Vendas/ID3/.json")

req = requests.get(f"{url}/Vendas/.json")
print(req.json())

{'-O2oo7Snwopj6oh0ucaz': {'cliente': 'Lira', 'produto': 'fone de ouvido', 'quantidade': 1, 'valor': 50}, 'ID1': {'cliente': 'Alon', 'produto': 'fone de ouvido', 'quantidade': 2, 'valor': 100}}
